In [348]:
#Code Prototype for 1D use on EXCLAIM Project with multiple position sensors whose time rates are different, code by Joaquin Matticoli

In [349]:
import numpy as np
import filterpy
from filterpy.kalman import KalmanFilter as kf
from filterpy.kalman import predict
from filterpy.kalman import update
from numpy.random import randn
#from filterpy.common import Q_discrete_white_noise
import matplotlib.pyplot as plt
#import filterpy.stats as stats
import math
from math import sqrt
import decimal
from decimal import *

In [350]:

np.random.seed(24)

def print_gh(predict, update, z1, z2, K):
    predict_template = '{: 7.3f} {: 8.3f}'
    update_template = '{:.3f}\t{:.3f}\t{: 7.3f} {: 7.3f}   {:.3f}'

    print(predict_template.format(predict[0], predict[1]),end='\t')
    print(update_template.format(z1, z2, update[0], update[1], K))

In [351]:
class PayloadSimulation(object):
    def __init__(self, x0=0., angular_velocity2=1.,
                 measurement1_var=0.0, measurement2_var = 0.0,
                 process_var=0.0, t_0 = 0.):
        """ x0 : initial orientation
            angular_velocity: (+=counterclockwise, -=clockwise)
            measurement_var: variances in measurement deg^2
            process_var: variance in process (deg/s)^2
        """
        self.x = x0
        self.angular_velocity = angular_velocity2
        self.meas1_std = sqrt(measurement1_var)
        self.meas2_std = sqrt(measurement2_var)
        self.process_std = sqrt(process_var)

    def move(self, dt):
        """Compute new orientation of the payload in dt seconds."""
        dx = self.angular_velocity + abs(randn()*self.process_std)
        self.x += dx * dt

    def sense_position(self, meas_std):
        """ Returns measurement of new orientation in degrees."""
        measurement = self.x + abs(randn()*meas_std)
        return measurement
    
#     def sense_velocity(self):
#         """ Returns measurement of new angular velocity in deg/sec."""
#         measurement2 = self.angular_velocity + abs(randn()*self.meas2_std)
#         return measurement2

    def move_and_sense(self, check, dt=1.0):
        """ Move payload, and return measurement of new orientation in degrees"""
        self.move(dt=dt)
        if check == 1:
            return self.sense_position(meas_std=self.meas1_std)
        if check == 2:
            return self.sense_position(meas_std=self.meas2_std)

In [352]:
printcase = 1 # If set to 1 it will print out the results
plotcase = 0 # If set to 1 it will plot results

# -----------USER INPUT STARTS------------------------------

number_of_variables = 2 #number of variables being measured
number_of_sensors = 1 #number of sensors used
process_var = Decimal('25.0') # variance in the payload's movement, idk what number is good for this??
sensor1_var = Decimal('2.0') # variance in the sensor
sensor2_var = Decimal('0.5') # variance in second sensor
x = 0. # guess of payload's initial orientation in deg
x_var = 100. # guess of payload's initial variance in deg^2
angular_velocity = Decimal('0.5') # angular velocity in deg/s
spectral_density = Decimal('0.2') #Spectral density of the measurement noise
dt = 1. #Time for process equations?
dt1 = Decimal('1.0') # time step in seconds for first sensor
dt2 = Decimal('0.3') # time step in seconds for second sensor
total_time = Decimal('75.0') # Total time passed from beginning to end of experiment
t = Decimal('0.0') #initial time in seconds

# dt1_copy = 1.0 # time step in seconds for first sensor
# dt2_copy = 0.3 # time step in seconds for second sensor

# -----------USER INPUT ENDS--------------------------------

smallest_time = min(dt1, dt2) #The smaller time increment
n_iter = math.floor(total_time/smallest_time)

    #NOTE: this code will assume that orientation is always increasing/decreasing and not sinusodial because a more complext Kalman filter is needed for those cases
    
# simulate payload and get measurements
payload = PayloadSimulation(
    x0=x, 
    angular_velocity2=angular_velocity*dt, 
    measurement1_var=sensor1_var, 
    measurement2_var=sensor2_var,
    process_var=process_var

# creates list of measurements, NOTE: modify this section if measurements are provided
zs = np.zeros()
zs2 = np.zeros()
ztrue = np.zeros()
check = 0
while t <= total_time:
    r1 = t % dt1
    r2 = t % dt2
    if r1 == 0. and r2 == 0.: #takes measurment of both sensors but only moves once
        zs.append(payload.move_and_sense(check=1, dt=dt1))
        zs2.append(payload.sense_position(meas_std=sqrt(sensor2_var)))
        t = t + smallest_time
    elif r1 < smallest_time and check == 0:
        t = t - r1
        zs.append(payload.move_and_sense(check=1, dt=dt1))
        t = t + r1
        check = 1
    else:
        zs2.append(payload.move_and_sense(check=2, dt=dt2))
        t = t + smallest_time
        check = 0
    ztrue.append(payload.x)
    t = t + smallest_time
finalposition = payload.x

SyntaxError: invalid syntax (<ipython-input-352-62fb1b105c1e>, line 40)

In [322]:
def plotstuff():
    plt.figure()
    plt.plot(zs,'k+',label='Noisy Position Measurements')
    #plt.plot(x1,'r*',label='A Priori Estimate')
    plt.plot(xhat,'b-',label='A Posteri Estimate')
    plt.plot(ztrue,color='g',label='True Value')
    plt.legend()
    plt.title('Estimate vs. Iteration Step', fontweight='bold')
    plt.xlabel('Iteration')
    plt.ylabel('Orientation (Degrees)')
    
    plt.figure()
    valid_iter = range(1,total_time) # Pminus not valid at step 0
    plt.plot(valid_iter,Pminus[valid_iter],label='A Posteri Error Estimate')
    plt.title('Estimated $\it{\mathbf{a \ priori}}$ Error vs. Iteration Step', fontweight='bold')
    plt.xlabel('Iteration')
    plt.ylabel('$(Position)^2$')
    #plt.setp(plt.gca(),'ylim',[0,.01])
    plt.show()

x1 = [] #Estimate before update
Pminus = [] #Variance after update
xhat = [] #Estimate after update
counter = 0
if printcase == 1:
    print('PREDICT\t\t\t\t\tUPDATE')
    print('     x      var\t\t  z1\t  z2\t    x      var      K')

# perform Kalman filter on measurement(s) z
f = kf(dim_x=number_of_variables, dim_z=number_of_sensors)
f.F = np.array([[1., 1], [0., 1.]]) #correct for calculating position from position and velocity
f.H = np.array([[1., 0.], [0., 1.]]) #this is for first sensor measuring position and second one velocity
f.x = np.array([[x], [angular_velocity*dt]])
f.Q *= np.array([[(dt**3)/3, (dt**2)/2],
                 [(dt**2)/2,  dt      ]]) *spectral_density
f.P = np.diag([x_var,process_var]) #Variance
f.R = np.diag([sensor1_var, sensor2_var])  #Noise in sensors

for i in range(total_time):    
    f.predict()
    prior = gaussian(f.x_prior[0,0], f.P_prior[0,0])
    f.update(np.array([zs[i], zs2[i]]))
    x = gaussian(f.x[0,0], f.P[0,0])
    if printcase == 1:
        print_gh(prior, x, zs[i], zs2[i], f.K[0,0])
    if plotcase == 1:
        x1.append = prior.mean
        xhat.append = x.mean
        Pminus.append = x.var

if printcase == 1:
    print()
    print('Final estimate:           {:10.3f}'.format(x.mean))
    print('Actual final orientation: {:10.3f}'.format(payload.x))
if plotcase == 1:
    plotstuff()

PREDICT					UPDATE
     x      var		  z1	  z2	    x      var      K


TypeError: 'float' object cannot be interpreted as an integer